# Statistics to Use (Ideas)
- Political - ?
- Population Density
- Migration patterns
- Age demographics
- Culture - ?
- Interest Rates
- GDP
- Number of housing units
- Proximity to social/metro centres
- City Age -- History (how would we quantify?) 
- Imports/Exports - ?
- Main Industry/Sector - Resources, raw materials
- Average Income
- Number of Schools, Education
- Geographic Location
- Number of highways, transporation infrastructure
- Terrain 

## Economic
Interest Rates, GDP, Imports/Exports, Average Income

## Important
- <b>Politics</b>
    - Partisan segration, polarity
- <b>Education</b>
- Abundance of housing units - ?
- City Age/History
- <b>Main Industry Sector</b>
- <b> Economic Measures</b>
- <b>Geographic Location</b>
    - Proximity to other cities, metro centres
- Number of highways, transporation infrastructure

# TO-DO:
- Add more features, fix poverty data

In [2]:
# Import Necessary Libraries
import numpy as np 
import pandas as pd

# FEATURE 1: RATIO OF PEOPLE TAKING PUBLIC TRANSPORATION TO WORK

transportation_data = pd.read_csv('../datasets/transportation_data/ACSST5Y2010.S0802_data_with_overlays_2020-03-04T134515.csv')

# Filter data to get only appropriate metropolitan areas
transportation_data = transportation_data[~transportation_data.NAME.str.contains('United State')]
num_public_transport = transportation_data['S0802_C04_001E'].to_numpy()[1:].astype(int)
num_workers = transportation_data['S0802_C01_001E'].to_numpy()[1:].astype(int)
ratio = np.divide(num_public_transport, num_workers)

# Standardisation, Feature Scaling
ratio = (ratio - np.mean(ratio))/(ratio.std())
ratio = [float(r) for r in ratio]

# Create new DataFrame with each metro area and corresponding attribute values
new_df = pd.DataFrame(zip(transportation_data['NAME'].to_numpy()[1:],  ratio), columns = ['metro_name', 'ratio_of_public_transport']).set_index('metro_name').sort_values('metro_name')
new_df['ratio_of_public_transport'].fillna(0, inplace = True)
new_df

AttributeError: 'Series' object has no attribute 'to_numpy'

In [ ]:


# FEATURE 2: MEDIAN HOUSEHOLD INCOME

income_data = pd.read_csv('../datasets/income_data/ACSST5Y2010.S2503_data_with_overlays_2020-03-04T163518.csv')
income_data = income_data[~income_data.NAME.str.contains('United States')]
median_income = income_data['S2503_C01_013E'].to_numpy()[1:].astype(int)

# Standardise
median_income = (median_income - np.mean(median_income))/(median_income.std())

# Make sure every element of the median_income column is a float
new_df['median_income'] = [float(m) for m in median_income]
new_df

In [ ]:
# FEATURE 3: PERCENT BELOW POVERTY LEVEL

#### CURRENTLY NOT WORKING #####

poverty_data = pd.read_csv('../datasets/poverty_data/ACSST1Y2010.S1701_data_with_overlays_2020-03-04T183351.csv')
poverty_data = poverty_data[~poverty_data.NAME.str.contains('United States')]
percentages = poverty_data['S1701_C03_001E'].to_numpy()[1:].astype(float)
percentages = (percentages - np.mean(percentages)) / np.std(percentages)

poverty_df = pd.DataFrame(zip(poverty_data['NAME'][1:], percentages),
             columns = ['metro_name', 'pctg_below_poverty_line']).sort_values('metro_name')

new_df2 = new_df.merge(poverty_df, on = 'metro_name', how='outer', suffixes=('', '_y')).fillna(0)
new_df2                 

In [ ]:
# FEATURE 4: POPULATION OF PEOPLE 3 AND OVER ENROLLED IN SCHOOL

school_data = pd.read_csv('../datasets/school_data/ACSST5Y2010.S1401_data_with_overlays_2020-03-04T192940.csv')

# Log-transform, standardise
enrolled = school_data['S1401_C01_001E'].to_numpy()[1:].astype(int)
enrolled = (enrolled - np.mean(enrolled)) / np.std(enrolled)

# Correct type, save to test DataFrame
new_df2['enrolled_in_school'] = [float(e) for e in enrolled]
new_df2.to_csv('test_01.csv')
new_df2

In [73]:
political_df = pd.read_csv('../geographic_eda/countypres_2000-2016.csv')
political_df = political_df[#(political_df['year'] == 2004) &
                            ((political_df['party'] == 'democrat')
                           | (political_df['party'] == 'republican'))] #keep only dem and rep parties
political_df.drop(
    ['version', 'state_po', 'state', 'office', 'candidate', 'totalvotes'], 
    axis=1, inplace=True)
political_df.dropna(subset=['FIPS'], axis=0, inplace=True)
political_df.rename(columns={'county': 'CountyName'}, inplace=True)
political_df['CountyName'] = political_df['CountyName'].apply(lambda x: str(x))

print(political_df['party'].unique())
political_df.head()

['democrat' 'republican']


,year,CountyName,FIPS,party,candidatevotes
0,2000,Autauga,1001.0,democrat,4942.0
1,2000,Autauga,1001.0,republican,11993.0
4,2000,Baldwin,1003.0,democrat,13997.0
5,2000,Baldwin,1003.0,republican,40872.0
8,2000,Barbour,1005.0,democrat,5188.0


In [78]:
cw = pd.read_csv('../datasets/CountyCrossWalk_Zillow.csv')
cw.head()

,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,FIPS,CBSACode
0,Pike,Pennsylvania,42,103,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",280,394913.0,42103,35620.0
1,Bronx,New York,36,5,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",401,394913.0,36005,35620.0
2,Essex,New Jersey,34,13,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",504,394913.0,34013,35620.0
3,Kings,New York,36,47,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",581,394913.0,36047,35620.0
4,Ocean,New Jersey,34,29,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",659,394913.0,34029,35620.0


In [80]:
cw = cw[['CountyName', 'CBSACode']]
cw['CountyName'] = cw['CountyName'].apply(lambda x: str(x))
cw.head()

,CountyName,CBSACode
0,Pike,35620.0
1,Bronx,35620.0
2,Essex,35620.0
3,Kings,35620.0
4,Ocean,35620.0


In [89]:
political_df.merge(cw, left_on='CountyName', right_on='CountyName', how='left')

,year,CountyName,FIPS,party,candidatevotes,CBSACode
0,2000,Autauga,1001.0,democrat,4942.0,33860.0
1,2000,Autauga,1001.0,republican,11993.0,33860.0
2,2000,Baldwin,1003.0,democrat,13997.0,19300.0
3,2000,Baldwin,1003.0,democrat,13997.0,33300.0
4,2000,Baldwin,1003.0,republican,40872.0,19300.0
5,2000,Baldwin,1003.0,republican,40872.0,33300.0
6,2000,Barbour,1005.0,democrat,5188.0,NaN
7,2000,Barbour,1005.0,democrat,5188.0,NaN
8,2000,Barbour,1005.0,republican,5096.0,NaN
9,2000,Barbour,1005.0,republican,5096.0,NaN
